In [2]:
import requests
import time
import numpy as np
import json

# Utils for timing

In [23]:
def average_duration(request: str, data_gen=str, method:str='GET', iterations: int=30000):
    '''
        Returns duration of the requests in s.
    '''

    if method == 'GET':
        rec_function = requests.get
    elif method == 'POST':
        rec_function = requests.post
    else:
        raise ValueError('Method has not been recognized!')
    
    durations = []
    for _ in range(iterations):
        start = time.time()
        rec_function(url=request, data=next(data_gen), headers={'Content-type': 'application/json'})
        end = time.time()

        durations.append(end - start)

    print('On average request takes: ', np.mean(durations))


def average_duration_alteration(request: str, extraction_request: str, method:str='PATCH', iterations: int=30000):
    '''
        Funtion for testing UPDATE, DELETE
    '''

    if method == 'PATCH':
        action = requests.patch
    elif method == 'DELETE':
        action = requests.delete
    else:
        raise ValueError('This method is not recognized!')

    extracted_data = requests.get(url=extraction_request)
    entries = json.loads(extracted_data.content)

    ids = [entry['telegram_id'] for entry in entries]
    durations = []

    for _ in range(iterations):
        for id in ids:
            start = time.time()
            action(
                url=request + str(id)
            )
            end = time.time()

            durations.append(end - start)

    print('On average request takes: ', np.mean(durations))


def generate_user(num: int):
    start = 9281497

    for i in range(num):
        data = json.dumps({
            'telegram_id' : start + i,
            'username': 'varenik',
            'about': '''On Unix, return the current processor time as a floating point number expressed in seconds. The precision, and in fact the very definition of the meaning of “processor time”, depends on that of the C function of the same name. On Windows, this function returns wall-clock seconds elapsed since the first call to this function, as a floating point number, based on the Win32 function QueryPerformanceCounter(). The resolution is typically better than one microsecond.'''
        })

        yield data


def generate_empty(num: int):
    for _ in range(num):
        yield json.dumps({})

# Tests of NoSQL: MongoDB

## Insert

In [ ]:
request = 'http://localhost:5052/users/insert'
iteration_num = 30000

average_duration(request=request, data_gen=generate_user(iteration_num), method='POST', iterations=iteration_num)

## Extract

In [7]:
request = 'http://localhost:5052/users'
iteration_num = 300

average_duration(request=request, data_gen=generate_empty(iteration_num), method='POST', iterations=iteration_num)

On average request takes:  0.0019246681531270345


## Update

In [25]:
request = 'http://localhost:5052/users/patch/'
extraction = 'http://localhost:5052/users'
iteration_num = 10

average_duration_alteration(request=request, extraction_request=extraction, method='PATCH', iterations=iteration_num)

On average request takes:  0.0027388036251068115


## Delete

In [ ]:
request = 'http://localhost:5052/users/delete/'
extraction = 'http://localhost:5052/users'
iteration_num = 10

average_duration_alteration(request=request, extraction_request=extraction, method='DELETE', iterations=iteration_num)

# Tests of SQL: PostgreSQL

## Insert

In [23]:
request = 'http://localhost:5051/users/insert'
requests_num = 30000

average_duration(request=request, data_gen=iter(generate_user(requests_num)), method='POST', iterations=requests_num)

On average request takes:  0.003915661104520162


## Extract

In [27]:
request = 'http://localhost:5051/users'
requests_num = 300

average_duration(request=request, data_gen=iter(generate_empty(requests_num)), method='GET', iterations=requests_num)

On average request takes:  0.15078813552856446


## Update

## Delete

# Results